### Problem
Solve the classic diet model.

In [1]:
# First, import packages
import pandas as pd
from IPython.display import Image 
import gurobipy as gp
from gurobipy import GRB

# Define a gurobipy model for the decision problem
m = gp.Model('diet')

Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-29


### Data

In [2]:
# Food categories, minimum nutrition and maximum nutrition information.

categories, minNutrition, maxNutrition = gp.multidict({
        "calories": [1800, 2200],
        "protein": [91, GRB.INFINITY],
        "fat": [0, 65],
        "sodium": [0, 1779],
})

In [3]:
type(categories)

gurobipy.tuplelist

In [4]:
# Food and cost information

foods, cost = gp.multidict({
        "hamburger": 2.49,
        "chicken": 2.89,
        "hot dog": 1.50,
        "fries": 1.89,
        "macaroni": 2.09,
        "pizza": 1.99,
        "salad": 2.49,
        "milk": 0.89,
        "ice cream": 1.59,
})

In [5]:
# Nutrition values for the foods

nutritionValues = {
    ("hamburger", "calories"): 410,
    ("hamburger", "protein"): 24,
    ("hamburger", "fat"): 26,
    ("hamburger", "sodium"): 730,
    ("chicken", "calories"): 420,
    ("chicken", "protein"): 32,
    ("chicken", "fat"): 10,
    ("chicken", "sodium"): 1190,
    ("hot dog", "calories"): 560,
    ("hot dog", "protein"): 20,
    ("hot dog", "fat"): 32,
    ("hot dog", "sodium"): 1800,
    ("fries", "calories"): 380,
    ("fries", "protein"): 4,
    ("fries", "fat"): 19,
    ("fries", "sodium"): 270,
    ("macaroni", "calories"): 320,
    ("macaroni", "protein"): 12,
    ("macaroni", "fat"): 10,
    ("macaroni", "sodium"): 930,
    ("pizza", "calories"): 320,
    ("pizza", "protein"): 15,
    ("pizza", "fat"): 12,
    ("pizza", "sodium"): 820,
    ("salad", "calories"): 320,
    ("salad", "protein"): 31,
    ("salad", "fat"): 12,
    ("salad", "sodium"): 1230,
    ("milk", "calories"): 100,
    ("milk", "protein"): 8,
    ("milk", "fat"): 2.5,
    ("milk", "sodium"): 125,
    ("ice cream", "calories"): 330,
    ("ice cream", "protein"): 8,
    ("ice cream", "fat"): 10,
    ("ice cream", "sodium"): 180,
}

### Decision Variable

In [6]:
# Create decision variables for the foods to buy
buy = m.addVars(foods, vtype = GRB.CONTINUOUS, name="buy")
m.update()
buy

# Another method:
# You could use Python looping constructs and m.addVar() to create these decision variables instead.
# The following would be equivalent
#
# buy = {}
# for f in foods:
#   buy[f] = m.addVar(name=f)

{'hamburger': <gurobi.Var buy[hamburger]>,
 'chicken': <gurobi.Var buy[chicken]>,
 'hot dog': <gurobi.Var buy[hot dog]>,
 'fries': <gurobi.Var buy[fries]>,
 'macaroni': <gurobi.Var buy[macaroni]>,
 'pizza': <gurobi.Var buy[pizza]>,
 'salad': <gurobi.Var buy[salad]>,
 'milk': <gurobi.Var buy[milk]>,
 'ice cream': <gurobi.Var buy[ice cream]>}

### Constraint

In [7]:
# Nutrition constraints
c1 = m.addConstrs(gp.quicksum(nutritionValues[f, c] * buy[f] for f in foods)
                  == [minNutrition[c], maxNutrition[c]] for c in categories)
m.update()
c1

# Another method:
#m.addConstrs(
#    (
#        gp.quicksum(nutritionValues[f, c] * buy[f] for f in foods)
#        == [minNutrition[c], maxNutrition[c]]
#        for c in categories
#    ),
#    "_",
#)
#m.update()

{'calories': <gurobi.Constr R0>,
 'protein': <gurobi.Constr R1>,
 'fat': <gurobi.Constr R2>,
 'sodium': <gurobi.Constr R3>}

### Objective Function

In [9]:
# The objective is to minimize the costs
# m.setObjective(buy.prod(cost), GRB.MINIMIZE)

# Using looping constructs, the preceding statement would be:
m.setObjective(gp.quicksum(buy[f]*cost[f] for f in foods), GRB.MINIMIZE)

# Another method:
#m.setObjective(buy.prod(cost), GRB.MINIMIZE)

In [12]:
# Save model
m.write("diet.lp")

### Solve/Optimize

In [13]:
# Solve
m.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Linux Mint 21.3")

CPU model: AMD Ryzen 5 5600G with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 4 rows, 12 columns and 39 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [9e-01, 3e+00]
  Bounds range     [6e+01, 2e+03]
  RHS range        [6e+01, 2e+03]

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.182886111e+01


### Print solution

In [11]:
def printSolution():
    if m.status == GRB.OPTIMAL:
        print(f"\nCost: {m.ObjVal:g}")
        print("\nBuy:")
        for f in foods:
            if buy[f].X > 0.0001:
                print(f"{f} {buy[f].X:g}")
    else:
        print("No solution")

printSolution()


Cost: 11.8289

Buy:
hamburger 0.604514
milk 6.97014
ice cream 2.59132


----------------------------------------------